In [2]:
import os

os.getcwd()

'c:\\Users\\oraio\\OneDrive\\Desktop\\AI-ML\\projects\\Computer Vision\\E2E_CV_Class_ChickenDisease\\research'

In [3]:
os.chdir("../")
os.getcwd()

'c:\\Users\\oraio\\OneDrive\\Desktop\\AI-ML\\projects\\Computer Vision\\E2E_CV_Class_ChickenDisease'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


#=====================================================================================================================#
#=====================================================================================================================#

from ChickenDisease.constants import *
from ChickenDisease.utils.common_functions import *


class ConfigurationManager:
    # create a constructor
    # create the artifacts_root folder "artifacts" described in config.yaml which is the root folder for all other data stages artifacts
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    # create the "data_ingestion" folder inside the "artifacts" folder, to store all the data ingestion artifacts
    #   - read the data_ingestion section from config.yaml
    # create and return the DataIngestionConfig dataclass object "data_ingestion_config"
    #   - define the attributes of the DataIngestionConfig dataclass from the data_ingestion section of config.yaml
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(root_dir = config.root_dir,
                                                    source_URL = config.source_URL,
                                                    local_data_file = config.local_data_file,
                                                    unzip_dir = config.unzip_dir)
        return data_ingestion_config

#=====================================================================================================================#
#=====================================================================================================================#

import os
import urllib.request as request
import zipfile
from ChickenDisease import logger
from ChickenDisease.utils.common_functions import get_size
#from ChickenDisease.entity.config_entity import DataIngestionConfig
from pathlib import Path


# create the DataIngestion class which will do the real work of downloading and extracting the data
#  - constructor should take the DataIngestionConfig object as input
#  - create the download_file method to download the file from source_URL to local_data_file
#  - create the extract_zip_file method to extract the zip file into the unzip_dir
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL, 
                                                     filename=self.config.local_data_file)
            logger.info(f"Downloaded file: {filename} with headers: {headers}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file} with size: {get_size(self.config.local_data_file)} bytes")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_dir_path = self.config.unzip_dir
        os.makedirs(name=unzip_dir_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zipped_content:
            zipped_content.extractall(path=unzip_dir_path)
            logger.info(msg=f"Extracted files to: {unzip_dir_path}")   


In [ ]:
# This is the first stage of the data ingestion pipeline for the Chicken Disease Classification project.
# It handles downloading the dataset from a specified URL and extracting it from a zip file.
# The `DataIngestionTrainingPipeline` class orchestrates the data ingestion process by initializing the configuration,

try:
    config = ConfigurationManager() 
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-08-30 23:47:43,865: INFO: common_functions: yaml file: config\config.yaml loaded successfully]
[2025-08-30 23:47:43,871: INFO: common_functions: yaml file: params.yaml loaded successfully]
[2025-08-30 23:47:43,873: INFO: common_functions: created directory at: artifacts]
[2025-08-30 23:47:43,874: INFO: common_functions: created directory at: artifacts/data_ingestion]
[2025-08-30 23:51:00,404: INFO: 3575629471: Downloaded file: artifacts/data_ingestion/data.zip with headers: Connection: close
Content-Length: 24090578
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2b48f3ad00c86862507771e4158ef3ea290cfbdd8a2822abc0ec347563d3fbc0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8EB6:1C8228:2F0678:354213:68B35561
Accept-Ranges: bytes
Date: Sat, 30 Aug 2025 19:47:46 GMT
Via: 1.1 varnish
X-